In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
cd '/content/drive/MyDrive/Colab Notebooks/Twitter Sentiment Analysis'

/content/drive/MyDrive/Colab Notebooks/Twitter Sentiment Analysis


In [3]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot
from scipy.stats import mannwhitneyu
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [13]:
def civalue(x):
    mx=np.mean(x)
    ci=np.zeros((2))
    ci[0]=mx-1.96*x.std()/math.sqrt(len(x))
    ci[1]=mx+1.96*x.std()/math.sqrt(len(x))
    return ci

def crosscorr(x,n):
    sef=np.zeros((n))
    for i in range(0,n):
        sef[i]=i+1
    for i in range(0,x.shape[1]-1):
        for j in range(0,x.shape[1]-1):
            if (np.array(np.where(sef==i)).shape[1]>0 and np.array(np.where(sef==j)).shape[1]>0):
                if i!=j and (x[i,j]>=0.7 or x[i,j]<=-0.7):
                    sef[np.array(np.where(sef==j))]=0;       
    return sef 

In [14]:
fileloc='smote outputs/folder/'
from scipy import stats

In [15]:
def sigtetfinal(j):
    fname=fileloc+str(j)+'n.csv'
    df=np.genfromtxt(fname,delimiter=',')
    y=df[:,-1]
    in0=np.where(y==0)
    in1=np.where(y==1)
    #in2=np.where(y==3)
    #in3=np.where(y==4)
    #in4=np.where(y==5)
    #in5=np.where(y==6)
    p=np.zeros((df.shape[1]-1))
    for i in range(0,df.shape[1]-1):
        fv=df[:,i]
        #w,p[i]=stats.f_oneway(fv[in0],fv[in1],fv[in2],fv[in3],fv[in4],fv[in5])
        w,p[i]=stats.f_oneway(fv[in0],fv[in1])
    return p,df

In [16]:
def coranaf(df,sigfn):
    corr=np.zeros((sigfn.shape[0],sigfn.shape[0]))
    for i in range(0,sigfn.shape[0]):
        for j in range(0,sigfn.shape[0]):
            a=np.corrcoef(df[:,sigfn[i]],df[:,sigfn[j]])
            corr[i,j]=a[0,1]
    sg=crosscorr(corr,sigfn.shape[0])
    return sg

In [21]:
for i in range(11,16):
    p,df=sigtetfinal(i)
    sigf=np.where(p<=0.05)
    data=np.concatenate((df[:,sigf[0]],df[:,-1].reshape(-1,1)), axis=1)
    fname=fileloc+str(i)+"mmm1.csv"
    np.savetxt(fname,data, delimiter=',', fmt='%f')
    sg=coranaf(df,sigf[0])
    sg=np.where(sg>0)
    s=sigf
    s=s[0]
    s=s[sg[0]]
    data=np.concatenate((df[:,s],df[:,-1].reshape(-1,1)), axis=1)
    fname=fileloc+str(i)+"mmm2.csv"
    np.savetxt(fname,data, delimiter=',', fmt='%f')

In [23]:
from sklearn.decomposition import PCA
for i in range(11,16):
    fname=fileloc+str(i)+'n.csv'
    df=np.genfromtxt(fname,delimiter=',')
    pca = PCA(n_components=10)
    pca.fit(df[:,0:-1])
    data=pca.transform(df[:,0:-1])
    data=np.concatenate((data,df[:,-1].reshape(-1,1)), axis=1)
    fname=fileloc+str(i)+"mmm3.csv"
    np.savetxt(fname,data, delimiter=',', fmt='%f')